Example of the process of doing DEMs for all orbits for a given observation of an AR – where we don't window out any intervals because of shifts, etc. Instead, we will save energy-specific information about the percent of NuSTAR emission in the chosen region as a function of time, for later inspection. This will help identify cases where pointing shifts might cause unphysical distortion to the DEM inputs. 

Note: auto-download of an AIA file (for use in the initial co-alignmnet) will break while the JSOC is still down. We will add a thing where you can point to an existing AIA file instead.

Overview:

- Define orbits
- Run time interval selection
- Examine resulting intervals
- Manually establish a co-alignment shift between NuSTAR and AIA
- Automatically find co-alignment shifts + make regions for all other time intervals (note: this relies on the assumption that the COM is a good representation of the location of the brightest source, i.e. that the NuSTAR data is primarially one blob).
- Save AIA region files for NCCS input
- NOT IN THIS NOTEBOOK: YOU THEN TAKE THOSE AND MAKE AIA INPUTS ON THE NCCS
- Conduct AIA/NuSTAR DEMs as a function of time, given all the above
- Plot results.
- Print some stats about "left out" times.

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import glob
# from astropy.io import fits
# from astropy import units as u
# import importlib
# import pathlib

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

# #import nustar_dem_prep as nu
import initial_analysis as ia
import orbit_auto as oa
import time_interval_selection as tis
import nustar_utilities as nuutil
import gauss2D as g2d


# import os
# # For macOS
# def play_sound():
#     os.system("afplay /System/Library/Sounds/Glass.aiff")

import pickle
import pathlib
import importlib


In [ ]:
key = '06-jun-20'


In [ ]:
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']


In [ ]:
ARDict

In [ ]:
for id in id_dirs:
    evt_data, hdr = ia.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

After TIS:

In [ ]:
importlib.reload(oa)

all_time_intervals, all_time_intervals_list = oa.find_all_intervals(working_dir)

#allpercentsA, allpercentsB = oa.check_region_emission(all_time_intervals, working_dir, grade='0_4', plot=True)
allpercentsA, allpercentsB = oa.check_region_emission(all_time_intervals, working_dir, grade='0_4', plot=True, efilter=[2.5,3.5])
allpercentsA, allpercentsB = oa.check_region_emission(all_time_intervals, working_dir, grade='0_4', plot=True, efilter=[3.5,6.])
allpercentsA, allpercentsB = oa.check_region_emission(all_time_intervals, working_dir, grade='0_4', plot=True, efilter=[6.,10.])

In [ ]:
import glob

#Set path to obsid directory - initial pipeline should have been run already.
ind=0
datapath=id_dirs[ind]
obsid=obsids[ind]

evt_data, hdr = ia.return_submap(datapath=datapath, fpm='A', return_evt_hdr=True)
time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
timerange = [time0.tt.datetime, time1.tt.datetime]
from datetime import timezone
timerange = [t.replace(tzinfo=timezone.utc) for t in timerange]

#Comment second line if you're not using this same example nustar orbit
#Edit it to include only the desired time interval (default- all times in file) once you've run this once
#timerange=[]
#timerange=[datetime.datetime(2018, 5, 29, 22, 22), datetime.datetime(2018, 5, 29, 23, 20)]

evtA = glob.glob(datapath+'/event_cl/*A06_cl.evt')
evtB = glob.glob(datapath+'/event_cl/*B06_cl.evt')
hkA  = glob.glob(datapath+'/hk/*A_fpm.hk')
hkB  = glob.glob(datapath+'/hk/*B_fpm.hk')

import lightcurves as lc

importlib.reload(lc)
lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[2.,4.], 
                              livetime_corr=False, save_dir=working_dir)
lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[4.,6.], 
                              livetime_corr=False, save_dir=working_dir)
lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[6.,10.], 
                              livetime_corr=False, save_dir=working_dir)

lc.plot_nustar_lightcurves(eranges = [[2.,4.],[4.,6.],[6.,10.]],
                           timerange=timerange, save_dir=working_dir)

In [ ]:
# importlib.reload(oa)
# oa.check_conseq(working_dir)

In [ ]:
aia_dir = '/Users/jmdunca2/sample_aia/sample_aia/'
import sunpy.map
import glob
from aiapy.calibrate.util import get_correction_table, get_pointing_table
from aiapy.calibrate import register, update_pointing, degradation, estimate_error

aiamaps = []
for id in id_dirs:
    print(id)
    evt_data, hdr = ia.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    start = str(time0)
    files = glob.glob(aia_dir+'*94A_'+start[0:10]+'T'+start[11:13]+'*')
    print(start)
    print(files)
    amap=sunpy.map.Map(files[0])
    #ptab = get_pointing_table(amap.date - 12 * u.h, amap.date + 12 * u.h)
    #m_temp = update_pointing(amap, pointing_table=ptab)
    try:
        m = register(amap)
    except TypeError:
        amap.meta.pop('crpix1')
        amap.meta.pop('crpix2')
        print('CRPIX issue on ', files)
        m = register(amap)

    aiamaps.append(m)

In [ ]:
importlib.reload(oa)
import pickle

orbit_ind=0

time_interval = all_time_intervals[orbit_ind][0]
nushift=[-10, 50]

#(first run)
dict = oa.nu_aia_coalign(time_interval, working_dir, nushift, save_dict=True, input_aia=aiamaps[orbit_ind])

time=time_interval
timestring = time[0].strftime('%H-%M-%S')
stopstring = time[1].strftime('%H-%M-%S')
timestring=timestring+'_'+stopstring
file=working_dir+timestring+'/'+timestring+'_aia_region.pickle'
try:
    with open(file, 'rb') as f:
        data = pickle.load(f)
    dict = oa.nu_aia_coalign(time_interval, working_dir, nushift, save_dict=True, input_aia = data['map'])
except FileNotFoundError: 
    print('what')


In [ ]:
importlib.reload(oa)
lead_intervals=[]
for at in all_time_intervals:
    lead_intervals.append(at[0])

reference_interval=time_interval


oa.coalign_based_on_prior(lead_intervals, working_dir, reference_interval, input_aias=aiamaps)

In [ ]:
#IF YOU WANT TO REVISE THE STARTING SHIFT SOMEWHERE OTHER THAN JUST AT THE VERY FIRST ORBIT - NEXT 2 CELLS

# importlib.reload(oa)
# import pickle

# orbit_ind=3

# time_interval = all_time_intervals[orbit_ind][0]
# nushift=[10, -50]

# #(first run)
# dict = oa.nu_aia_coalign(time_interval, working_dir, nushift, save_dict=True, input_aia=aiamaps[orbit_ind])

# time=time_interval
# timestring = time[0].strftime('%H-%M-%S')
# stopstring = time[1].strftime('%H-%M-%S')
# timestring=timestring+'_'+stopstring
# file=working_dir+timestring+'/'+timestring+'_aia_region.pickle'
# try:
#     with open(file, 'rb') as f:
#         data = pickle.load(f)
#     dict = oa.nu_aia_coalign(time_interval, working_dir, nushift, save_dict=True, input_aia = data['map'])
# except FileNotFoundError: 
#     print('what')


In [ ]:
# importlib.reload(oa)
# lead_intervals=[]
# for at in all_time_intervals[3:]:
#     lead_intervals.append(at[0])

# reference_interval=time_interval

# oa.coalign_based_on_prior(lead_intervals, working_dir, reference_interval, input_aias=aiamaps)

In [ ]:
importlib.reload(oa)
suborbit_dirs = oa.make_all_aia_dicts(all_time_intervals, working_dir, key)
print(set(suborbit_dirs))